# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 2. Preprocessing (전처리)
- 수집한 데이터 : 요리이름, 카테고리, 난이도, 소요시간, 재료, 요리방법, 요리설명, 링크, 이미지링크
- 전처리할 데이터 : 재료에 섞인 불용어 위주로 제거


### 라이브러리 import

In [1]:
# 데이터 분석
import numpy as np
import pandas as pd

# 진행시간 표시
import time
from tqdm.notebook import tqdm
tqdm.pandas()

# 정규표현식
import re

### 데이터 불러오기

In [2]:
# 크롤링한 데이터 불러오기
raw_recipes = pd.read_csv('data/raw_recipes.csv')

In [3]:
raw_recipes.shape

(4340, 9)

In [4]:
raw_recipes.head(1)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,"['꼬시래기', '시판용 냉면육수', '오이', '삶은 달걀', '쌈무', '통깨'...","['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ..."


### 전처리 진행하기

In [5]:
# 수기로 발견한 대표적인 불용어와 많은 레시피에 두루 포함되는 조미료 계열을 리스트 형태로 변수 할당
stop_words = ['송송','썬','삶은','짤주머니','사각','X','곱게','리덕션','채썬',
              '냉장','늙은','판','미니','집','흰','두절','레드','모양','같이','잘','익은',
              '고운','익힌','주머니','건세모가사리','빨강','썰은','소분한','작은',
              '마른','벗긴','밥심','뜨거운','따듯한','씨를','크기로','제거','생물','빨간',
              '갈은','지은','데친','국물용','부순','꼬치','찬','차가운','빨간','중하',
              '컬러','구멍','따뜻한','A','틀','초밥용','몽글몽귤','튜브','두꺼운','크기','모둠',
              '해감한','와','구운','굵은','그린','굵게','익은','시판','것','없는','cm','상온',
              '삶은','무첨가','이지잇','잘게', '자른','깐','찐','씻은','잡채용',
              '고운', '면수']

In [6]:
# 텍스트 전처리 (1차)
def preprocessing(text):
    '''
    문자열을 전달하면 불용어, 중복, 공백을 제거하여 리스트 형태로 반환하는 함수

    Args:
    - 문자열 (예 : "['순살 고등어', '청주(또는 맛술)', '소금', '소금']")

    Returns:
    - 리스트 (예 : ['순살 고등어', '청주', '소금'])

    Example:
    >>> preprocessing("['순살 고등어', '청주(또는 맛술)', '소금', '소금']")
    ['순살고등어', '청주', '소금']
    '''
    process = eval(text)

    process = [elem for elem in process if not re.search(r'\*', elem)]
    process = [elem.replace('\xa0', '') for elem in process]
    process = [elem.replace(' ', '') for elem in process]

    process = ' '.join(set(process))
    process = re.sub(r'\([^)]*\)', '', process).strip()
    process = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', process).strip()
    pattern = '|'.join(stop_words)
    process = re.sub(pattern, '', process)
    process = re.sub(r'또는', ' ', process)

    process = process.split(' ')

    process = [elem for elem in process if elem != '']
    
    return process

In [7]:
# 데이터 프레임 복사
df = raw_recipes.copy()

In [8]:
# 재료 컬럼에 텍스트 전처리 (1차) 적용
df['재료'] = df['재료'].progress_map(preprocessing)
df['재료']

  0%|          | 0/4340 [00:00<?, ?it/s]

0       [통깨, 꼬시래기, 용냉면육수, 쌈무, 매실청, 참기름, 고춧가루, 소금, 달걀, ...
1                  [통깨, 참기름, 닭가슴살, 찹쌀밥, 다진당근, 소금, 물, 통마늘]
2       [양조식초, 통깨, 밥, 깻잎, 고추장, 다진마늘, 참기름, 설탕, 상추, 양파, ...
3       [후춧가루, 올리고당, 땅콩가루, 굴소스, 다진마늘, 돼지등갈비, 통후추, 생강가루...
4       [후춧가루, 파스타면, 케이퍼, 마늘, 루꼴라, 레몬제스트, 소금, 앤초비, 올리브오일]
                              ...                        
4335    [후춧가루, 대파, 청양고추, 다진마늘, 설탕, 홍고추, 신김치, 멸치다시마육수, ...
4336    [후춧가루, 통깨, 대파, 밥, 닭다리살, 굴소스, 올리고당, 꽈리고추, 다진마늘,...
4337    [후춧가루, 체다슬라이스치즈, 베이컨, 감자, 소금, 플레인요거트, 파슬리가루, 올...
4338    [생강가루, 전분가루, 소금, 케첩, 순살고등어, 땅콩, 맛술, 간장, 물, 고추장...
4339    [후춧가루, 디종머스타드, 파슬리, 코코넛밀크, 소금, 타임, 두유, 대파, 뉴트리...
Name: 재료, Length: 4340, dtype: object

In [9]:
# 확인하기
df.head(2)

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,"[통깨, 꼬시래기, 용냉면육수, 쌈무, 매실청, 참기름, 고춧가루, 소금, 달걀, ...","['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ..."
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,https://static.wtable.co.kr/image/production/s...,"[통깨, 참기름, 닭가슴살, 찹쌀밥, 다진당근, 소금, 물, 통마늘]","['냄비에 물과 닭가슴살, 마늘, 대파를 넣고 15분간 삶아주세요. ', '삶은 닭...",아플 때나 기운이 없을 때 간단히 끓여 먹기 좋은 닭죽! 맛이 좋은 건 물론 속을 ...


In [10]:
# 재료수 파생변수를 생성하는 함수
def ingredients_count(text):
    ing_count = len(text)
    return ing_count

In [11]:
# 재료 컬럼에 재료수 파생변수 생성
df['재료수'] = df['재료'].progress_map(ingredients_count)
df.head(2)

  0%|          | 0/4340 [00:00<?, ?it/s]

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,"[통깨, 꼬시래기, 용냉면육수, 쌈무, 매실청, 참기름, 고춧가루, 소금, 달걀, ...","['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ...",12
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,https://static.wtable.co.kr/image/production/s...,"[통깨, 참기름, 닭가슴살, 찹쌀밥, 다진당근, 소금, 물, 통마늘]","['냄비에 물과 닭가슴살, 마늘, 대파를 넣고 15분간 삶아주세요. ', '삶은 닭...",아플 때나 기운이 없을 때 간단히 끓여 먹기 좋은 닭죽! 맛이 좋은 건 물론 속을 ...,8


In [12]:
# 텍스트 전처리 (2차)
def preprocessing2(list):
    '''
    리스트를 전달하면 공백으로 구분된 하나의 문자열로 반환하는 함수

    Args:
    - 리스트 (예 : ['순살 고등어', '청주', '소금'])

    Returns:
    - 문자열 (예 : "순살고등어 청주 소금")

    Example:
    >>> preprocessing2(['순살 고등어', '청주', '소금'])
    "순살고등어 청주 소금"
    '''
    process = str(list)
    process = eval(process)
    process = ' '.join(set(process))

    return process

In [13]:
# 재료 컬럼에 텍스트 전처리 (2차) 적용
df['재료'] = df['재료'].progress_map(preprocessing2)
df.head(2)

  0%|          | 0/4340 [00:00<?, ?it/s]

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,https://static.wtable.co.kr/image/production/s...,통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀,"['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\r\n(tip...","꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 ...",12
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,https://static.wtable.co.kr/image/production/s...,통깨 참기름 닭가슴살 찹쌀밥 다진당근 소금 물 통마늘,"['냄비에 물과 닭가슴살, 마늘, 대파를 넣고 15분간 삶아주세요. ', '삶은 닭...",아플 때나 기운이 없을 때 간단히 끓여 먹기 좋은 닭죽! 맛이 좋은 건 물론 속을 ...,8


In [14]:
# csv 파일로 저장하고 불러오기
df.to_csv('data/preprocessed_recipes.csv', index=False)
n_df = pd.read_csv('data/preprocessed_recipes.csv')

In [15]:
# 불용어 제거 확인하기 
filtered_df = n_df[n_df['재료'].apply(lambda x: any(word in x for word in stop_words))]
filtered_df 

,요리,종류,난이도,소요시간,링크,사진,재료,요리방법,설명,재료수


In [16]:
# 텍스트 전처리 전과 후를 비교하는 함수
def sample_test(origin_df, processed_df, column_name):
    sample = origin_df.sample(1)[column_name]
    processed_sample = processed_df.iloc[list(sample.index)][column_name]
    with pd.option_context('display.max_colwidth', None):
        return display(sample.to_string(index=False), processed_sample.to_string(index=False))

In [17]:
# 실행할 때 마다 sample 무작위로 변경하면서 확인 가능
sample_test(raw_recipes, df, '재료')

"['고등어', '삶은 무우', '삶은 통연근', '삶은 통우엉', '생강편', '파채', '설탕', '고춧가루', '정종', '미림', '양조간장']"

'고등어 생강편 통연근 무우 설탕 미림 고춧가루 양조간장 정종 파채 통우엉'